###General Instructions
In this assignment, you will need to complete the code samples where indicated to accomplish the given objectives. **Be sure to run all cells** and export this notebook as an HTML with results included.  Upload the exported HTML file to Canvas by the assignment deadline.

####Assignment
Unlike previous exercises, you will not be provided any sample code from which to work.  You will be given some very high-level instructions and are expected to figure out a solution from there.

The MNIST-Fashion dataset is a collection of 60,000 training and 10,000 test images identifying fashion products in 10 categories: 

| Label        | Description          |
| ------------- |:-------------:|
| 0	| T-shirt/top| 
| 1	| Trouser| 
| 2| 	Pullover| 
| 3	| Dress| 
| 4	| Coat| 
| 5	| Sandal| 
| 6	| Shirt| 
| 7	| Sneaker| 
| 8 | 	Bag| 
| 9	| Ankle boot| 

Build a CNN consisting of the following layers to classify images to a particular category:

1. 2D convolutional layer leveraging 64 nodes and using a 3x3 filter
2. MaxPooling layer with a 2x2 kernel
3. Flattening layer 
4. Dense layer consisting of 96 nodes
5. Dense output layer with 10 nodes

Use standard activation functions with each convolutional and dense layer. Be sure to apply a 25% dropout after the pooling and first dense layer. 

Retrieve your data using the [built-in fashion_mnist dataset](https://keras.io/datasets/#fashion-mnist-database-of-fashion-articles) found within Keras.  Be sure to apply appropriate data transformations. 

Adjust the number of epochs to achieve a > 90% accuracy. Try both the Adadelta and the Stochastic Gradient Descent optimization algorithms (with a learning rate of 0.01 as your starting point) with a loss function of categorical_crossentropy when you compile. Stop whenever gives you the required result.

Do not use Horovod. Be sure to run the last cell of this assignment so that predictions are presented along side sample images.

HINT: Review the first lab on CNNs for the patterns to follow. You will want to go with a lower learning rate than used in that lab or your model can get stuck.  Ideally, it should take you between 10 and 20 epochs to reach the required accuracy. With the Stochastic Gradent Descent optimizer, you might try bumping up your momentum to 0.9 to speed up its learning.

In [4]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adadelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

/databricks/python/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping, MutableMapping
Using TensorFlow backend.

In [5]:
# retrieve dataset and apply transformations
from keras.datasets import fashion_mnist

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

Downloading data from http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
 8192/29515 [=======>......................] - ETA: 0s16384/29515 [===============>..............] - ETA: 0s32768/29515 [=================================] - 0s 5us/step
Downloading data from http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
 8192/26421880 [..............................] - ETA: 0s 16384/26421880 [..............................] - ETA: 4:16 49152/26421880 [..............................] - ETA: 2:51 106496/26421880 [..............................] - ETA: 1:53 229376/26421880 [..............................] - ETA: 1:10 466944/26421880 [..............................] - ETA: 43s  950272/26421880 [>.............................] - ETA: 25s 1916928/26421880 [=>............................] - ETA: 14s 3489792/26421880 [==>...........................] - ETA: 8s  5062656/26421880 [====>.........................] - ETA: 5s 6643712/26421880 [======>.......................] - ETA: 4s 8216576/26421880 [========>.....................] - ETA: 3s 9797632/26421880 [==========>...................] - ETA: 3s11370496/26421880 [===========>..................] - ETA: 2s12951552/26421880 [=============>................] - ETA: 2s14524416/26421880 [===============>..............] - ETA: 1s16105472/26421880 [=================>............] - ETA: 1s17678336/26421880 [===================>..........] - ETA: 1s19251200/26421880 [====================>.........] - ETA: 1s19259392/26421880 [====================>.........] - ETA: 1s20832256/26421880 [======================>.......] - ETA: 0s22405120/26421880 [========================>.....] - ETA: 0s23986176/26421880 [==========================>...] - ETA: 0s25559040/26421880 [============================>.] - ETA: 0s26427392/26421880 [==============================] - 4s 0us/step
Downloading data from http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz
8192/5148 [===============================================] - 0s 0us/step
Downloading data from http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz
 8192/4422102 [..............................] - ETA: 0s 16384/4422102 [..............................] - ETA: 43s

In [6]:
print('The X_train dataset is organized as a {0} with a shape of {1}'.format(type(X_train), X_train.shape))
print('The X_test dataset is organized as a {0} with a shape of {1}'.format(type(X_test), X_test.shape))
print('The y_train dataset is organized as a {0} with a shape of {1}'.format(type(y_train), y_train.shape))
print('The y_test dataset is organized as a {0} with a shape of {1}'.format(type(y_test), y_test.shape))

The X_train dataset is organized as a <class 'numpy.ndarray'> with a shape of (60000, 28, 28)
The X_test dataset is organized as a <class 'numpy.ndarray'> with a shape of (10000, 28, 28)
The y_train dataset is organized as a <class 'numpy.ndarray'> with a shape of (60000,)
The y_test dataset is organized as a <class 'numpy.ndarray'> with a shape of (10000,)

In [7]:
X_train.shape[0]

Out[5]: 60000

In [8]:
# get shape of original images
img_rows, img_cols = X_train[0].shape[:2]

# restructure arrays for (rows, columns, channels)
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

# record new input shape for layers
input_shape = (img_rows, img_cols, 1)

# normalize the data
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

# turn y into categorical 
y_train_cat = keras.utils.to_categorical(y_train, 10)
y_test_cat = keras.utils.to_categorical(y_test, 10)

In [9]:
# construct & compile CNN
model = Sequential()

# Convolution Layer
model.add(
  Conv2D(32, 
         kernel_size=(3, 3),
         activation='relu',
         input_shape=input_shape)
         ) 


# Pooling with stride (2, 2)
model.add(
  MaxPooling2D(
    pool_size=(2, 2)
    )
  )
# 25% dropout
model.add(Dropout(0.25))

# flatten out result
model.add(Flatten())

model.add(Dense(96, activation='relu'))
model.add(Dropout(0.5))

# Apply Softmax for the classes 0 through 9 (10 classes total)
model.add(Dense(10, activation='softmax'))

# Loss function (crossentropy) and Optimizer (Adadelta)
model.compile(
  loss=keras.losses.categorical_crossentropy,
optimizer=keras.optimizers.Adadelta(),
  metrics=['accuracy']
  )


In [10]:
# train the CNN
model.fit(
      X_train,
      y_train_cat,
      batch_size=128,
      epochs=15,
      verbose=1,
      validation_data=(X_test, y_test_cat)
      )



Train on 60000 samples, validate on 10000 samples
Epoch 1/15
 128/60000 [..............................] - ETA: 9:33 - loss: 2.3024 - acc: 0.1172 256/60000 [..............................] - ETA: 5:01 - loss: 2.2418 - acc: 0.1758 384/60000 [..............................] - ETA: 3:30 - loss: 2.2200 - acc: 0.1719 512/60000 [..............................] - ETA: 2:44 - loss: 2.1601 - acc: 0.2168 640/60000 [..............................] - ETA: 2:17 - loss: 2.1145 - acc: 0.2391 768/60000 [..............................] - ETA: 1:58 - loss: 2.0713 - acc: 0.2604 896/60000 [..............................] - ETA: 1:45 - loss: 2.0235 - acc: 0.2801 1024/60000 [..............................] - ETA: 1:35 - loss: 1.9720 - acc: 0.3047 1152/60000 [..............................] - ETA: 1:28 - loss: 1.9331 - acc: 0.3186 1280/60000 [..............................] - ETA: 1:22 - loss: 1.8761 - acc: 0.3383 1408/60000 [..............................] - ETA: 1:17 - loss: 1.8305 - acc: 0.3558 1536/60000 [..............................] - ETA: 1:13 - loss: 1.8005 - acc: 0.3672 1664/60000 [..............................] - ETA: 1:09 - loss: 1.7795 - acc: 0.3750 1792/60000 [..............................] - ETA: 1:06 - loss: 1.7472 - acc: 0.3856 1920/60000 [..............................] - ETA: 1:04 - loss: 1.7226 - acc: 0.3927 2048/60000 [>.............................] - ETA: 1:01 - loss: 1.6903 - acc: 0.4067 2176/60000 [>.............................] - ETA: 59s - loss: 1.6571 - acc: 0.4214  2304/60000 [>.............................] - ETA: 58s - loss: 1.6397 - acc: 0.4258 2432/60000 [>.............................] - ETA: 56s - loss: 1.6203 - acc: 0.4313 2560/60000 [>.............................] - ETA: 55s - loss: 1.6035 - acc: 0.4359 2688/60000 [>.............................] - ETA: 53s - loss: 1.5840 - acc: 0.4431 2816/60000 [>.............................] - ETA: 52s - loss: 1.5628 - acc: 0.4496 2944/60000 [>.............................] - ETA: 51s - loss: 1.5390 - acc: 0.4575 3072/60000 [>...........

In [11]:
# score the CNN
score = model.evaluate(X_test, y_test_cat, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.25764170660972596
Test accuracy: 0.9062

In [12]:
# visualize results
items = {
  0:'T-Shirt/Top',
  1:'Trouser',
  2:'Pullover', 
  3:'Dress',
  4:'Coat',
  5:'Sandal', 
  6:'Shirt',
  7:'Sneaker', 
  8:'Bag',
  9:'Ankle Boot'  
  }

plt.figure(figsize=(25,5))

for i, y in enumerate(y_test[0:5]):
  
  yhat = model.predict(X_test[i].reshape(1,28,28,1))
  
  s = plt.subplot(1,5,i+1)
  s.set_title('item = {0} (prob = {1:.2%})'.format( items[np.where(yhat == np.amax(yhat))[1][0]], np.amax(yhat)))
  plt.imshow(X_test[i][:,:,0], cmap='gray') 
  
display()